In [ ]:
# pip install --upgrade openai

In [11]:
import os
import time
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
from openai import OpenAI
from typing import Any, List, Union, Dict

In [4]:
client = OpenAI(
    api_key=os.environ['GPT_APIKEY']
)

def get_gpt_response(prompt, model='gpt-4o-mini'):
  response = client.responses.create(model="gpt-4o-mini", input=prompt)
  return response.output_text

In [5]:
class Symptom:
    def __init__(self, rus_name: str, curr_val: Any, avail_vals: Union[List[Any], Dict]):
        self.russian_name = rus_name
        self.current_value = curr_val
        self.available_values = avail_vals

class Prompt:
    def __init__(self):

        self.age = Symptom('возраст', None, [str(age) + ' лет' for age in range(18, 101)])
        self.gender = Symptom('пол', None, {'женский': 0, 'мужской': 1})
        self.cheastpaintype = Symptom('боль в груди', None, {'типичная стенокардия': 0, 'атипичная стенокардия': 1, 'неангинозная боль': 2, 'бессимптомный': 3})
        self.restingbp = Symptom('артериальное давление в состоянии покоя', None, list(range(70, 230)))
        self.cholesterol = Symptom('повышенный ли холестерин', None, {'нет': 0, 'да': 1})
        self.cholesterol_int = Symptom('уровень холестерина (в мг/дл)', None, list(range(126, 565)))
        self.restingecg = Symptom('результаты электрокардиографии в состоянии покоя', None, {
            'норма': 0,
            'наличие аномалии зубца ST-T (инверсия зубца T и/или подъем или снижение ST > 0,05 мВ)': 1,
            'демонстрация вероятной или определенной гипертрофии левого желудочка по критериям Эстеса': 2
        })
        self.maxhr = Symptom('максимальная достигнутая частота сердечных сокращений', None, list(range(70, 240)))
        self.fastingbs = Symptom('уровень сахара в крови натощак больше 120 mg/dl', None, {'нет': 0, 'да': 1})
        self.exerciseangina = Symptom('имеется стенокардия, вызванная физической нагрузкой', None, {'нет': 0, 'да': 1})
        self.oldpeak = Symptom('есть снижение ST сегмента электрокардиограммы вызванный физической нагрузкой', None, {'нет': 0, 'да': 1})
        self.st_slope = Symptom('наклон сегмента ST при пиковой нагрузке', None, {
            'восходящий': 0,
            'плоский': 1,
            'нисходящий': 2
        })
        self.nummajorvessels = Symptom('количество крупных сосудов , окрашенных с помощью флюороскопии', None, list(range(0, 5)))
        self.thal = Symptom('результат таллиевого стресс-теста', None, {
            'неизвестно': 0,
            'фиксированный дефект': 1,
            'норма': 2,
            'обратимый дефект': 3
        })

        self.height = Symptom('рост (см)', None, list(range(150, 221)))
        self.weight = Symptom('вес (кг)', None, list(range(40, 201)))
        self.ap_hi = Symptom('верхнее давление', None, list(range(60, 240)))
        self.ap_lo = Symptom('нижнее давление', None, list(range(40, 150)))
        self.gluc = Symptom('повышенный уровень глюкозы', None, {'нет': 0, 'да': 1})
        self.smoke = Symptom('курит ли', None, {'нет': 0, 'да': 1})
        self.alco = Symptom('употребляет ли алкоголь', None, {'нет': 0, 'да': 1})
        self.active = Symptom('занимается ли физической активностью', None, {'нет': 0, 'да': 1})

        self.all_symptoms = [self.age, self.cheastpaintype, self.restingbp, self.cholesterol,
               self.fastingbs, self.restingecg, self.maxhr, self.exerciseangina,
               self.oldpeak, self.st_slope, self.nummajorvessels, self.thal,
               self.cholesterol_int, self.gender, self.height, self.weight,
               self.ap_hi, self.ap_lo, self.gluc, self.smoke, self.alco, self.active]

        self.N = None
        self.selected_symptoms = []

    def select_random_symptoms(self):
        self.N = random.randint(0, len(self.all_symptoms))

        for i in range(self.N):
            symp_obj = random.choice(self.all_symptoms)
            symp_rus_name = symp_obj.russian_name
            if isinstance(symp_obj.available_values, list):
                random_value = random.choice(symp_obj.available_values)
                symp_rus_value = random_value
                symp_value = random_value
                if symp_obj.russian_name == 'возраст':
                    symp_value = symp_value.split(' ')[0]  # избавляюсь от приписки " лет"

            else:
                random_value = random.choice(list(symp_obj.available_values.items()))
                symp_rus_value = random_value[0]
                symp_value = random_value[1]
            self.selected_symptoms.append((symp_obj, symp_rus_name, symp_rus_value, symp_value))

    def get_symptom_name(self, obj):
        for attr_name, value in self.__dict__.items():
            if value == obj:
                return attr_name

    def generate_prompt_text(self):
        result = {
            'prompt_text': '',
            'rus_symptoms': {},
            'symptoms': {}
        }

        for obj, rus_name, rus_value, value in self.selected_symptoms:
            eng_name = self.get_symptom_name(obj)
            result['symptoms'][eng_name] = value
            result['rus_symptoms'][rus_name] = rus_value

        rus_prompt = ', '.join([str(key) + ': ' + str(value) for key, value in result['rus_symptoms'].items()])
        result['prompt_text'] = 'Сгенерируй разнообразный текст жалобы пациента от первого лица без переноса строк, у которого есть следующие симптомы:\n' \
            + rus_prompt + \
        ' используй разные паттерны. Без лишних слов, только жалобы. Желательно использовать другие слова симптомов, чтобы был понятен контекст.'
        return result


In [13]:
N = 10  # количество запросов к gpt
data = []

for i in tqdm(range(N)):
    pr = Prompt()
    pr.select_random_symptoms()
    prompt_dict = pr.generate_prompt_text()
    prompt_text = prompt_dict['prompt_text']
    response = get_gpt_response(prompt=prompt_text)
    data.append({'user_prompt': response, 'symptoms': prompt_dict['symptoms']})
    time.sleep(22)  # у данной модели есть лимит 3 запроса в минуту на бесплатном тарифе

100%|██████████| 10/10 [04:13<00:00, 25.35s/it]


In [14]:
df = pd.DataFrame(data)
final_df = pd.concat([df['user_prompt'], pd.json_normalize(df['symptoms'])], axis=1)
print(final_df.shape)
final_df.head()

(10, 23)


,user_prompt,weight,exerciseangina,restingbp,st_slope,age,height,oldpeak,restingecg,nummajorvessels,...,gender,cholesterol,active,ap_lo,maxhr,cheastpaintype,thal,alco,smoke,cholesterol_int
0,"Я чувствую сильную одышку, когда поднимаюсь по...",136.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Меня беспокоит высокое артериальное давление, ...",159.0,0.0,223.0,2.0,23,165.0,1.0,0.0,4.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"У меня часто наблюдается сердцебиение, появила...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Я чувствую постоянное напряжение в груди, особ...",NaN,NaN,NaN,0.0,NaN,171.0,1.0,NaN,NaN,...,1.0,NaN,1.0,149.0,146.0,1.0,NaN,NaN,NaN,NaN
4,Я испытываю постоянную одышку и дискомфорт в г...,190.0,NaN,145.0,2.0,NaN,166.0,NaN,2.0,NaN,...,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [16]:
final_df.to_csv('symptoms_prompts_generated_data.csv', index=False)